Backup Database Azure Blob Storage
==================================

Description
-----------

Notebook to walk through Azure Storage configuration, key generation, and scripting of a backup to blob storage command for on-premises SQL Server Database.

Steps of this procedure include:
1. Connect to Azure subscription
2. Provision resource group for SQL VM migration
3. Create a storage account
4. Create container for Blobs
5. Get Azure storage account key
6. Get Azure storage container Uri
7. Create Sql credentials on Sql server instances
8. Backup database on specific instance of Sql Server
9. Remove Sql credentials which were created on Sql server instances

In [ ]:
#Set Parameters
$Location = ""                           # Specify the valid Location such as 'West US 2','EASTUS' etc...
$ResourceGroup = ""                      # Resource group name(It should be of alphabets case insensitive)
$SkuName = ""                            # Specify the valid SKU...Such as 'Standard_LRS'

$SqlPath = "sqlserver:\sql\$($env:COMPUTERNAME)"      #This script will generate Sql Path

### <b>Sign in to Azure </b>
Sign in to your Azure Subscription with the _Connect-AzAccount_ command and follow the on-screen directions.

In [ ]:
Connect-AzAccount

### <b>Get Subscription</b>
Below command will open a _**Dialouge Box**_ with list of subscriptions. Selecting one of those will set that Subscription for rest of the commands.

In [ ]:
$Subscription = Get-AzSubscription | Out-GridView -PassThru
Set-AzContext -SubscriptionName $Subscription

_If you don't know which Location you want to use, you can list the available Locations. Display the list of Locations by using the following code example and find the one you want to use. This example uses eastus. Store the Location in a variable and use the variable so you can change it in one place._

In [ ]:
Get-AzLocation | select Location
$Location


### <b>Create a resource group</b>
Create an Azure resource group with _New-AzResourceGroup_. A resource group is a logical container into which Azure resources are deployed and managed.

In [ ]:
# Create Azure resource group, if necessary
$ResourceGroup = Get-AzResourceGroup -Name $ResourceGroup

if (!$ResourceGroup)
{
    # Need to create a new resource group
    Write-Output "Resource Group $ResourceGroup does not exist. Creating..."
    $ResourceGroup = New-AzResourceGroup -Name $RG1 -Location $Location
}

### <b>Create a storage account</b>
Create a standard, general-purpose storage account with LRS replication by using _New-AzStorageAccount_. Next, get the storage account context that defines the storage account you want to use. When acting on a storage account, reference the context instead of repeatedly passing in the credentials. Use the following example to create a storage account called storageaccountazure with locally redundant storage (LRS) and blob encryption (enabled by default).

In [ ]:
$StorageAccountName = "storageaccountazure"
$StorageAccount = New-AzStorageAccount -ResourceGroupName $ResourceGroup  -Name $StorageAccountName  -SkuName $SkuName -Location $Location

$Ctx = $StorageAccount.Context

### <b>Create a container</b>
Blobs are always uploaded into a container. You can organize groups of blobs like the way you organize your files on your computer in folders.
Set the container name, then create the container by using _New-AzStorageContainer_. Set the permissions to blob to allow public access of the files. The container name in this example is quickstartblobs.

In [ ]:
$ContainerName = "quickstartblobs"
New-AzStorageContainer -Name $ContainerName -Context $Ctx -Permission blob

### <b>Get Azure Storage Acount Key</b>
This script will get the key for Storage Account which is been created.

In [ ]:
$StorageAccountKey = `
    (Get-AzStorageAccountKey `
    -ResourceGroupName $ResourceGroup `
    -Name $StorageAccountName).Value[0]

### <b>Get Azure Storage Container Uri</b>
The following script can be used to get the Uri of Storage container.

In [ ]:
    $StorageUri = (Get-AzStorageAccount -ResourceGroupName $ResourceGroup -Name $StorageAccountName | Get-AzStorageContainer | Where-Object { $_.Name -eq $AzureContainerName }).CloudBlobContainer.Uri.AbsoluteUri 

### <b>Create a SQL credential on all Instances of SQL Server </b>
The following script can be used to create a generic SQL credential on all the Instances of SQL Server on a computer. If there's already an existing credential with the same name on one of the Instances of the computer, the script shows the error and continues.

In [ ]:
set-executionpolicy remotesigned
# load the sqlps module
import-module sqlps  
  
# set parameters
$SecureString = ConvertTo-SecureString $StorageAccountKey -AsPlainText -Force  
$CredentialName = "myCredential-$(Get-Random)"

Write-Host "Generate credential: " $CredentialName
  
#cd to sql server and get Instances  
cd $SqlPath
$Instances = Get-ChildItem

#loop through Instances and create a SQL credential, output any errors
foreach ($Instance in $Instances)  {
    try {
        $Path = "$($SqlPath)\$($Instance.DisplayName)\credentials"
        New-SqlCredential -Name $CredentialName -Identity $StorageAccountName -Secret $SecureString -Path $Path -ea Stop | Out-Null
        Write-Host "...generated credential $($Path)\$($CredentialName)."  }
    catch { Write-Host $_.Exception.Message } }

### <b>Full backup for system Databases on a specific Instance of SQL Server </b>
The full script can be used to back up the master and the msdb Databases on a named Instance of SQL Server. The same script can be used for any Instance by changing the Instance parameter value. SQL Server's default Instance is named DEFAULT.

In [ ]:
set-executionpolicy remotesigned 
import-module sqlps  

$InstanceName = "DEFAULT"
$BackupUrlContainer = "https://$StorageAccountName.blob.core.windows.net/$ContainerName/"  

Write-Host "Backup Database: " $InstanceName " to " $BackupUrlContainer
  
cd "$($SqlPath)\$($InstanceName)"

#loop through Instance and backup specific Databases
$Databases = "master"  
foreach ($Database in $Databases) {
    try {
        Write-Host "...starting backup: " $Database
        Backup-SqlDatabase -Database $Database -BackupContainer $BackupUrlContainer -SqlCredential $CredentialName -Compression On
        Write-Host "...backup complete." }
    catch { Write-Host $_.Exception.Message } }

### <b>Remove a SQL credential from Instances of SQL Server </b>
The following script can be used to remove a specific credential from all the Instances of SQL Server installed on the computer. If the credential does not exist on a specific Instance, an error message is displayed, and the script continues until all Instances are checked.

In [ ]:
set-executionpolicy remotesigned 
import-module sqlps

Write-Host "Delete credential: " $CredentialName

cd $SqlPath
$Instances = Get-ChildItem

#loop through Instances and delete a SQL credential
foreach ($Instance in $Instances)  {
    try {
        $Path = "$($SqlPath)\$($Instance.DisplayName)\credentials\$($CredentialName)"
        Remove-SqlCredential -Path $Path -ea Stop | Out-Null
        Write-Host "...deleted credential $($Path)."  }
    catch { Write-Host $_.Exception.Message } }